# Virtual Piano Tutor

## Project Description

This is a notebook for tracking my progress on VPT...

- Best Classifier as of 11/30
    - SVM {'C': 100, 'gamma': 0.1, 'kernel': 'rbf'}

## TODO List

- TODAY
    - Decide on RDF model to keep for rest of project
    - Work on RDF data and annotations
    - Add results to file
    - Rewrite RDF for GridSearchCV
        - Extend RDF
    - Work on ideas for paper
        - Visualizations
    - Play with CAE
    - How to automate this...
    - Windowing/Summarizing
    
- DONE
    - ~~Organize RDF data~~
    - ~~Generate data from already extracted hands...~~
    - ~~Get notebook running on Compute Canada~~
    - ~~Get data on Compute Canada~~
    - ~~Setup CAE to deal with hand images~~
    - ~~setup data for training autoencoder on LH and RH~~
    - ~~Train Autoencoder for LH and Rh~~
    

- Bad Segmentation
    - p3c - left hand (not terrible)
    - p1s - right hand (shouldn't use)
    - p5a - Both could use some work but still caputures most of the left hand (RH not so good...)
    - p5c - not good (left hand passable...)
    
- Add noise to CAE
    - http://scikit-image.org/docs/dev/api/skimage.util.html#random-noise
    
- ~~Multiple Participants~~
    - ~~have one holdout set participant~~
        - ~~Test with p1&2 training p3 testing, then p1&3...~~
    - ~~have one holdout set exercise~~

- Test with RH too

- Windowing data
    - Summarize data for classification
    - Majority Voting (or with probabilities)

- Look for other features
    - Others??
    - ~~Autoencoder features~~
    - ~~HONV~~
    
- Work on hand segmentation
   - See p1e for bad examples
   - How to validate segmentation?
       - Statistical analysis on length and width ratios
       
- Visualize !!!
    - Input 
    - Results !!!
        - F Scores
        - Accuracy
        - Try weighted instead of macro




- Finish Project Description

- ~~Turn into functions~~
    
- ~~Verify Segmentation~~
    - have only done basic verification
    
- ~~FIRST THING: Test by ignoring training data (p1s) and then using train_test_split on recordings~~
    - ~~Data should be ready for spliting~~
    
- ~~Remove data from testing to find culprit~~
    
- ~~Track my progress better !!! (duh through notebooks!)~~

# Setup

## Libraries

In [1]:
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import cv2

from vpt.features.features import *
import vpt.utils.image_processing as ip
import vpt.settings as s
import vpt.hand_detection.depth_context_features as dcf

%load_ext autoreload
%autoreload 2

## Some helper functions

### Load/Save Data Set

In [2]:
def load_data(testing_p, M, radius, feature_type, data_type):
    base = "data/posture/extracted/"
    data_path = os.path.join(base, "{}_M{}_rad{:0.2f}_{}_".format(testing_p, M, radius, feature_type))
    data = np.load(data_path + data_type + "_data_combined.npz")    
    return data

## Project Setup

### Generate or Load Data

In [24]:
#### Load data for a single paricipant
M = 5
radius = .15
feature_type = "hog2"

data = load_data("all_participants", M, radius, feature_type, "train")
print("X LH", data["X_lh"].shape, "y LH", data["y_lh"].shape, data["vis_lhs"].shape)
print("X RH", data["X_rh"].shape, "y RH", data["y_rh"].shape, data["vis_rhs"].shape)
print("Filenames", data["filenames"].shape)

X LH (15818, 3600) y LH (15818,) (15818, 180, 180)
X RH (15818, 3600) y RH (15818,) (15818, 180, 180)
Filenames (15818,)


In [25]:
## using p6 for validation
r = re.compile('p3')
vmatch = np.vectorize(lambda x:bool(r.search(x)))
val_p = vmatch(data['filenames'])

X_lh = data['X_lh'][~val_p]
y_lh = data['y_lh'][~val_p]
X_rh = data['X_rh'][~val_p]
y_rh = data['y_rh'][~val_p]
filenames = data['filenames'][~val_p]


X_lh_test = data['X_lh'][val_p]
y_lh_test = data['y_lh'][val_p]
X_rh_test = data['X_rh'][val_p]
y_rh_test = data['y_rh'][val_p]
filenames_test = data['filenames'][val_p]

print("Cross Val Data:")
print(X_lh.shape, y_lh.shape)
print(X_rh.shape, y_rh.shape)
print(filenames.shape)
print()
print("Validation Data")
print(X_lh_test.shape, y_lh_test.shape)
print(X_rh_test.shape, y_rh_test.shape)
print(filenames_test.shape)

Cross Val Data:
(11246, 3600) (11246,)
(11246, 3600) (11246,)
(11246,)

Validation Data
(4572, 3600) (4572,)
(4572, 3600) (4572,)
(4572,)


In [26]:
groups = np.zeros_like(filenames, dtype=int)

for p in ["p1", "p3", "p4", "p6"]:
    p_num = int(p[1])
    groups[np.where(np.char.find(filenames, p) != -1)] = p_num
    
print(groups.shape)
print(np.unique(groups))
print(groups)

(11246,)
[1 4 6]
[1 1 1 ..., 6 6 6]


# Classification

### Libraries

In [27]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, LeaveOneGroupOut
from imblearn.pipeline import Pipeline

from sklearn.decomposition import PCA

from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN

## Load Data for Classification

In [28]:
## find all "static" data so we can ignore for now
r = re.compile('p[\d]s')

# remove p#s data
vmatch = np.vectorize(lambda x:bool(r.search(x)))
rem_static = vmatch(filenames)
rem_static_test = vmatch(filenames_test)

X_lh_train, y_lh_train, filenames_train, groups_train = X_lh[~rem_static], y_lh[~rem_static], filenames[~rem_static], groups[~rem_static]
X_rh_train, y_rh_train, filenames_train, groups_train = X_rh[~rem_static], y_rh[~rem_static], filenames[~rem_static], groups[~rem_static]

In [29]:
X_lh_val, y_lh_val, filenames_val = X_lh_test[~rem_static_test], y_lh_test[~rem_static_test], filenames_test[~rem_static_test]
X_rh_val, y_rh_val, filenames_val = X_rh_test[~rem_static_test], y_rh_test[~rem_static_test], filenames_test[~rem_static_test]

In [30]:
X_comb_train = np.vstack((X_lh_train, X_rh_train))
y_comb_train = np.hstack((y_lh_train, y_rh_train))
groups_comb_train = np.hstack((groups_train, groups_train))
filenames_comb_train = np.hstack((filenames_train, filenames_train))

X_comb_val = np.vstack((X_lh_val, X_rh_val))
y_comb_val = np.hstack((y_lh_val, y_rh_val))
filenames_comb_val = np.hstack((filenames_val, filenames_val))

In [31]:
print("Cross Validation Data")
print("LH:", X_lh_train.shape, y_lh_train.shape)
print(np.unique(y_lh_train, return_counts=True))
print("RH:", X_rh_train.shape, y_rh_train.shape)
print(filenames_train.shape)
print(groups_train.shape)
print(np.unique(y_rh_train, return_counts=True))
print("Combined:", X_comb_train.shape, y_comb_train.shape)
print(filenames_comb_train.shape)
print(groups_comb_train.shape)
print(np.unique(y_comb_train, return_counts=True))
print()
print()
print("Validation Data")
print("LH:", X_lh_val.shape, y_lh_val.shape)
print(np.unique(y_lh_val, return_counts=True))
print("RH:", X_rh_val.shape, y_rh_val.shape)
print(np.unique(y_rh_val, return_counts=True))
print("Combined:", X_comb_val.shape, y_comb_val.shape)
print(np.unique(y_comb_val, return_counts=True))
print(filenames_comb_val.shape)

Cross Validation Data
LH: (9629, 3600) (9629,)
(array([0, 1, 2]), array([6890, 1718, 1021]))
RH: (9629, 3600) (9629,)
(9629,)
(9629,)
(array([0, 1, 2]), array([7496, 2106,   27]))
Combined: (19258, 3600) (19258,)
(19258,)
(19258,)
(array([0, 1, 2]), array([14386,  3824,  1048]))


Validation Data
LH: (2650, 3600) (2650,)
(array([0, 1, 2]), array([1267, 1336,   47]))
RH: (2650, 3600) (2650,)
(array([0]), array([2650]))
Combined: (5300, 3600) (5300,)
(array([0, 1, 2]), array([3917, 1336,   47]))
(5300,)


## Model Testing

### SVM

In [11]:
## Parameters for SVMs
steps = [('SMOTE', SMOTE()), ("SVC", SVC())]
param_grid = [
  {'SVC__C': [1, 10, 100], 'SVC__kernel': ['linear'], 'SMOTE__kind': ['regular', 'borderline1', 'borderline2', 'svm']},
  {'SVC__C': [1, 10, 100], 'SVC__gamma': [.0001, .001, .01, .1], 'SVC__kernel': ['rbf'], 'SMOTE__kind': ['regular', 'borderline1', 'borderline2', 'svm']},
 ]

# steps = [('SMOTE', SMOTEENN()), ("SVC", SVC())]
# param_grid = [
#   {'SVC__C': [1, 10, 100, 1000], 'SVC__kernel': ['linear'], 'SMOTE__smote': [SMOTE(kind='regular'), SMOTE(kind='borderline1'), SMOTE(kind='borderline2'), SMOTE(kind='svm')]},
#   {'SVC__C': [1, 10, 100, 1000], 'SVC__gamma': [.0001, .001, .01, .1], 'SVC__kernel': ['rbf'], 'SMOTE__smote': [SMOTE(kind='regular'), SMOTE(kind='borderline1'), SMOTE(kind='borderline2'), SMOTE(kind='svm')]},
#  ]


pipeline = Pipeline(steps)

scores = ['f1']
logo = LeaveOneGroupOut()

In [16]:
# Hyper Parameter Tuning
for score in scores:
    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        print("## Tuning hyper-parameters for {}".format(score))
        print()

        if score is "accuracy":
            scoring = score
        else:
            scoring = '{}_macro'.format(score)      
        
        #### TRAIN COMBINED LH & RH
        clf_comb = GridSearchCV(pipeline, param_grid, cv=logo.get_n_splits(groups=groups_comb_train), scoring=scoring, n_jobs=2, verbose=2)
        clf_comb.fit(X_comb_train, y_comb_train)

        print("Best Combined Parameters set found on data set:")
        print()
        print(clf_comb.best_params_)
        print()
        print("Grid scores on data set:")
        print()
        means = clf_comb.cv_results_['mean_test_score']
        stds  = clf_comb.cv_results_['std_test_score']
        for mean, std, params in zip(means, stds, clf_comb.cv_results_['params']):
            print("%0.3f (+/-%0.3f) for %r" % (mean, std, params))
        print()

## Tuning hyper-parameters for f1

Fitting 3 folds for each of 60 candidates, totalling 180 fits
[CV] SMOTE__kind=regular, SVC__C=1, SVC__kernel=linear ...............
[CV] SMOTE__kind=regular, SVC__C=1, SVC__kernel=linear ...............
[CV]  SMOTE__kind=regular, SVC__C=1, SVC__kernel=linear, total= 1.8min
[CV] SMOTE__kind=regular, SVC__C=1, SVC__kernel=linear ...............
[CV]  SMOTE__kind=regular, SVC__C=1, SVC__kernel=linear, total= 3.0min
[CV] SMOTE__kind=regular, SVC__C=10, SVC__kernel=linear ..............
[CV]  SMOTE__kind=regular, SVC__C=1, SVC__kernel=linear, total= 2.2min
[CV] SMOTE__kind=regular, SVC__C=10, SVC__kernel=linear ..............
[CV]  SMOTE__kind=regular, SVC__C=10, SVC__kernel=linear, total= 2.4min
[CV] SMOTE__kind=regular, SVC__C=10, SVC__kernel=linear ..............
[CV]  SMOTE__kind=regular, SVC__C=10, SVC__kernel=linear, total= 2.3min
[CV] SMOTE__kind=regular, SVC__C=100, SVC__kernel=linear .............
[CV]  SMOTE__kind=regular, SVC__C=10, SVC__kernel

[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed: 158.5min


[CV]  SMOTE__kind=regular, SVC__C=1, SVC__gamma=0.0001, SVC__kernel=rbf, total=12.2min
[CV] SMOTE__kind=regular, SVC__C=1, SVC__gamma=0.001, SVC__kernel=rbf 
[CV]  SMOTE__kind=regular, SVC__C=1, SVC__gamma=0.0001, SVC__kernel=rbf, total=11.2min
[CV] SMOTE__kind=regular, SVC__C=1, SVC__gamma=0.001, SVC__kernel=rbf 
[CV]  SMOTE__kind=regular, SVC__C=1, SVC__gamma=0.001, SVC__kernel=rbf, total= 5.2min
[CV] SMOTE__kind=regular, SVC__C=1, SVC__gamma=0.001, SVC__kernel=rbf 
[CV]  SMOTE__kind=regular, SVC__C=1, SVC__gamma=0.001, SVC__kernel=rbf, total= 7.3min
[CV] SMOTE__kind=regular, SVC__C=1, SVC__gamma=0.01, SVC__kernel=rbf .
[CV]  SMOTE__kind=regular, SVC__C=1, SVC__gamma=0.001, SVC__kernel=rbf, total= 5.9min
[CV] SMOTE__kind=regular, SVC__C=1, SVC__gamma=0.01, SVC__kernel=rbf .
[CV]  SMOTE__kind=regular, SVC__C=1, SVC__gamma=0.01, SVC__kernel=rbf, total= 2.7min
[CV] SMOTE__kind=regular, SVC__C=1, SVC__gamma=0.01, SVC__kernel=rbf .
[CV]  SMOTE__kind=regular, SVC__C=1, SVC__gamma=0.01, SVC

[CV] SMOTE__kind=borderline1, SVC__C=10, SVC__gamma=0.01, SVC__kernel=rbf 
[CV]  SMOTE__kind=borderline1, SVC__C=10, SVC__gamma=0.001, SVC__kernel=rbf, total= 4.5min
[CV] SMOTE__kind=borderline1, SVC__C=10, SVC__gamma=0.01, SVC__kernel=rbf 
[CV]  SMOTE__kind=borderline1, SVC__C=10, SVC__gamma=0.01, SVC__kernel=rbf, total= 2.9min
[CV] SMOTE__kind=borderline1, SVC__C=10, SVC__gamma=0.01, SVC__kernel=rbf 
[CV]  SMOTE__kind=borderline1, SVC__C=10, SVC__gamma=0.01, SVC__kernel=rbf, total= 3.4min
[CV] SMOTE__kind=borderline1, SVC__C=10, SVC__gamma=0.1, SVC__kernel=rbf 
[CV]  SMOTE__kind=borderline1, SVC__C=10, SVC__gamma=0.01, SVC__kernel=rbf, total= 2.8min
[CV] SMOTE__kind=borderline1, SVC__C=10, SVC__gamma=0.1, SVC__kernel=rbf 
[CV]  SMOTE__kind=borderline1, SVC__C=10, SVC__gamma=0.1, SVC__kernel=rbf, total= 8.8min
[CV] SMOTE__kind=borderline1, SVC__C=10, SVC__gamma=0.1, SVC__kernel=rbf 
[CV]  SMOTE__kind=borderline1, SVC__C=10, SVC__gamma=0.1, SVC__kernel=rbf, total=10.4min
[CV] SMOTE__ki

[CV]  SMOTE__kind=borderline2, SVC__C=100, SVC__gamma=0.01, SVC__kernel=rbf, total= 2.7min
[CV] SMOTE__kind=borderline2, SVC__C=100, SVC__gamma=0.01, SVC__kernel=rbf 
[CV]  SMOTE__kind=borderline2, SVC__C=100, SVC__gamma=0.01, SVC__kernel=rbf, total= 3.2min
[CV] SMOTE__kind=borderline2, SVC__C=100, SVC__gamma=0.1, SVC__kernel=rbf 
[CV]  SMOTE__kind=borderline2, SVC__C=100, SVC__gamma=0.01, SVC__kernel=rbf, total= 2.8min
[CV] SMOTE__kind=borderline2, SVC__C=100, SVC__gamma=0.1, SVC__kernel=rbf 
[CV]  SMOTE__kind=borderline2, SVC__C=100, SVC__gamma=0.1, SVC__kernel=rbf, total= 9.5min
[CV] SMOTE__kind=borderline2, SVC__C=100, SVC__gamma=0.1, SVC__kernel=rbf 
[CV]  SMOTE__kind=borderline2, SVC__C=100, SVC__gamma=0.1, SVC__kernel=rbf, total=11.0min
[CV] SMOTE__kind=svm, SVC__C=1, SVC__gamma=0.0001, SVC__kernel=rbf ...
[CV]  SMOTE__kind=borderline2, SVC__C=100, SVC__gamma=0.1, SVC__kernel=rbf, total= 9.6min
[CV] SMOTE__kind=svm, SVC__C=1, SVC__gamma=0.0001, SVC__kernel=rbf ...
[CV]  SMOTE__k

[Parallel(n_jobs=2)]: Done 158 tasks      | elapsed: 725.6min


[CV]  SMOTE__kind=svm, SVC__C=10, SVC__gamma=0.0001, SVC__kernel=rbf, total=13.7min
[CV] SMOTE__kind=svm, SVC__C=10, SVC__gamma=0.001, SVC__kernel=rbf ...
[CV]  SMOTE__kind=svm, SVC__C=10, SVC__gamma=0.001, SVC__kernel=rbf, total= 7.3min
[CV] SMOTE__kind=svm, SVC__C=10, SVC__gamma=0.001, SVC__kernel=rbf ...
[CV]  SMOTE__kind=svm, SVC__C=10, SVC__gamma=0.001, SVC__kernel=rbf, total= 9.6min
[CV] SMOTE__kind=svm, SVC__C=10, SVC__gamma=0.01, SVC__kernel=rbf ....
[CV]  SMOTE__kind=svm, SVC__C=10, SVC__gamma=0.001, SVC__kernel=rbf, total= 7.7min
[CV] SMOTE__kind=svm, SVC__C=10, SVC__gamma=0.01, SVC__kernel=rbf ....
[CV]  SMOTE__kind=svm, SVC__C=10, SVC__gamma=0.01, SVC__kernel=rbf, total= 6.0min
[CV] SMOTE__kind=svm, SVC__C=10, SVC__gamma=0.01, SVC__kernel=rbf ....
[CV]  SMOTE__kind=svm, SVC__C=10, SVC__gamma=0.01, SVC__kernel=rbf, total= 6.2min
[CV] SMOTE__kind=svm, SVC__C=10, SVC__gamma=0.1, SVC__kernel=rbf .....
[CV]  SMOTE__kind=svm, SVC__C=10, SVC__gamma=0.01, SVC__kernel=rbf, total= 7.

[Parallel(n_jobs=2)]: Done 180 out of 180 | elapsed: 862.8min finished


Best Combined Parameters set found on data set:

{'SMOTE__kind': 'borderline2', 'SVC__C': 1, 'SVC__gamma': 0.001, 'SVC__kernel': 'rbf'}

Grid scores on data set:

0.591 (+/-0.061) for {'SMOTE__kind': 'regular', 'SVC__C': 1, 'SVC__kernel': 'linear'}
0.578 (+/-0.055) for {'SMOTE__kind': 'regular', 'SVC__C': 10, 'SVC__kernel': 'linear'}
0.575 (+/-0.055) for {'SMOTE__kind': 'regular', 'SVC__C': 100, 'SVC__kernel': 'linear'}
0.585 (+/-0.061) for {'SMOTE__kind': 'borderline1', 'SVC__C': 1, 'SVC__kernel': 'linear'}
0.576 (+/-0.055) for {'SMOTE__kind': 'borderline1', 'SVC__C': 10, 'SVC__kernel': 'linear'}
0.574 (+/-0.054) for {'SMOTE__kind': 'borderline1', 'SVC__C': 100, 'SVC__kernel': 'linear'}
0.599 (+/-0.080) for {'SMOTE__kind': 'borderline2', 'SVC__C': 1, 'SVC__kernel': 'linear'}
0.589 (+/-0.077) for {'SMOTE__kind': 'borderline2', 'SVC__C': 10, 'SVC__kernel': 'linear'}
0.585 (+/-0.080) for {'SMOTE__kind': 'borderline2', 'SVC__C': 100, 'SVC__kernel': 'linear'}
0.596 (+/-0.066) for {'SMOTE__

In [17]:
from sklearn.metrics import accuracy_score
y_comb_true, y_comb_pred = y_comb_val, clf_comb.predict(X_comb_val)

print("Comb Validatation Score:", accuracy_score(y_comb_true, y_comb_pred))
print("Comb Confustion Matrix:\n", confusion_matrix(y_comb_true, y_comb_pred))
print(classification_report(y_comb_true, y_comb_pred))
print()


Comb Validatation Score: 0.658113207547
Comb Confustion Matrix:
 [[2504 1362   51]
 [ 352  984    0]
 [  13   34    0]]
             precision    recall  f1-score   support

          0       0.87      0.64      0.74      3917
          1       0.41      0.74      0.53      1336
          2       0.00      0.00      0.00        47

avg / total       0.75      0.66      0.68      5300




### Single Classifier

In [ ]:
steps = [('PCA', PCA(n_components=1500)), ('SMOTE', SMOTE(kind="borderline2")), ("SVC", SVC(C=100, gamma=.0001, kernel='rbf', probability=False))]
# steps = [('SMOTE', SMOTE(kind="borderline2")), ("SVC", SVC(C=1, gamma=.001, kernel='rbf', probability=False))]
pipeline = Pipeline(steps)
pipeline.fit(X_comb_train, y_comb_train)

In [ ]:
y_comb_true, y_comb_pred = y_comb_val, pipeline.predict(X_comb_val)
print("Comb Validatation Score:", accuracy_score(y_comb_true, y_comb_pred))
print("Comb Confustion Matrix:\n", confusion_matrix(y_comb_true, y_comb_pred))
print(classification_report(y_comb_true, y_comb_pred))

In [41]:
window_size = 30
y_comb_true_maj = []
for i in range(0, y_comb_true.size, window_size):
    u, counts = np.unique(y_comb_true[i:i+window_size], return_counts=True)
    pred = u[np.argmax(counts)]
    y_comb_true_maj.append(pred)
    
y_comb_true_maj = np.array(y_comb_true_maj)

In [42]:
y_comb_pred_maj = []
for i in range(0, y_comb_pred.size, window_size):
    u, counts = np.unique(y_comb_pred[i:i+window_size], return_counts=True)
    pred = u[np.argmax(counts)]
    y_comb_pred_maj.append(pred)
    
y_comb_pred_maj = np.array(y_comb_pred_maj)

In [43]:
print("rH Validatation Score:", accuracy_score(y_comb_true_maj, y_comb_pred_maj))
print("rH Confustion Matrix:\n", confusion_matrix(y_comb_true_maj, y_comb_pred_maj))

rH Validatation Score: 0.71186440678
rH Confustion Matrix:
 [[97 34  0]
 [16 29  0]
 [ 1  0  0]]
